Pre-Test Window: 2020-03-24 to 2020-04-06        
Test Period: 2020-04-10 to 2020-04-23        
Test Segments: B, J          
Control Segments: D, G            

In [4]:
import warnings
warnings.filterwarnings('ignore',)

import importlib
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#Initialize Package
import pmutils.creds as pu
from pmstats.abtest import *
from pmstats.plots import *
from pmstats.lightspeed import *
from pmstats.utils import *

#Other packages
import pandas as pd
import numpy as np

from datetime import date, timedelta

In [5]:
my_conn = pu.create_connection()

In [6]:
#versions=['3.22','3.22.01','3.22.02']
# versions=None
# print(versions)
test_segments = ['B','J']
control_segments = ['D','G']

pre_start_date = '2020-03-24'
pre_end_date = '2020-04-06'
post_start_date = '2020-04-10'
post_end_date = '2020-04-23'

In [7]:
north_star_metrics = [
    #number of buyers out of eligible group on the same day they receive the email
    'buyer_conversion_rate'
    #'orders_buy_now_200andmore'
    #,'gmv_buy_now_200andmore'
    #,'aov_buy_now_200andmore'
]

business_metrics = [
    #whole segment
    'gmv'
    ,'orders'
    ,'aov'
    ,'buyers'
    ,'gmv - buy now'
    ,'orders - buy now'
    ,'aov - buy now'
    ,'buyers - buy now'
    ,'gmv - buyer offer'
    ,'orders - buyer offer'
    ,'aov - buyer offer'
    ,'buyers - buyer offer'
    
    ,'au'
    ,'gmv per au'
    #,'gmv_200andmore'
    #,'orders_200andmore'
]


business_metrics_other = [
    #business metrics of eligible users
    'eligible_users'
    ,'eligible_buyers'
    ,'eligible_gmv'
    ,'eligible_buy_now_gmv'
    ,'eligible_buyer_offer_gmv'
    ,'eligible_orders'
    ,'eligible_buy_now_orders'
    ,'eligible_buyer_offer_orders'
    ,'eligible_aov'
    ,'eligible_buy_now_aov'
    ,'eligible_buyer_offer_aov'
    ,'eligible_buy_now_buyers'
    ,'eligible_buyer_offer_buyers']

    
    #clicks
#     ,'buy now clicks'
#     ,'buy now clickers'
#     ,'buy confirmation views'
#     ,'buy confirmation viewers'
#     ,'submit order clicks'
#     ,'submit order clickers'                 
#     ,'buyer offer clicks'
#     ,'buyer offer clickers']


test_metrics = north_star_metrics + business_metrics  + business_metrics_other

In [8]:
#NORTH STAR - Daily Buyer Conversion Rate + eligible user group metrics
eligible_group = dict(
name = 'eligible_group',
sql = """

select eligible_date,
       user_id,
       o.buyer_id,
       o.order_id,
       date(o.booked_at) as book_date,
       o.order_gmv,
       o.offer_id,
       offer_initiated_by
FROM analytics_scratch.yc_abandon_eligible ace
         LEFT JOIN
     (SELECT o1.*, of.offer_initiated_by
      FROM analytics.dw_orders o1
               LEFT JOIN analytics.dw_offers of ON o1.offer_id = of.offer_id
      WHERE is_valid_order IS TRUE) o
     ON ace.user_id = o.buyer_id
         and (date(o.booked_at) = eligible_date or date(o.booked_at) is null)
  
""",
join = "eligible_group.user_id = exposure.user_id AND eligible_group.eligible_date = exposure.event_date",
metric_list = [
    "COUNT(DISTINCT eligible_group.user_id) AS eligible_users"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date IS NOT NULL THEN eligible_group.user_id END) AS eligible_buyers"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date IS NOT NULL THEN eligible_group.order_id END) AS eligible_orders"
    ,"SUM(CASE WHEN eligible_group.book_date is not null THEN order_gmv*0.01 ELSE 0 END) AS eligible_gmv"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date IS NOT NULL AND eligible_group.offer_id IS NULL THEN eligible_group.user_id END) AS eligible_buy_now_buyers"
    ,"SUM(CASE WHEN eligible_group.book_date is not null AND eligible_group.offer_id IS NULL THEN order_gmv*0.01 END) AS eligible_buy_now_gmv"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date is NOT NULL AND eligible_group.offer_id IS NULL THEN eligible_group.order_id END) AS eligible_buy_now_orders"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date IS NOT NULL AND eligible_group.offer_id IS NOT NULL AND offer_initiated_by = 'buyer' THEN eligible_group.user_id END) AS eligible_buyer_offer_buyers"
    ,"SUM(CASE WHEN eligible_group.book_date is not null AND eligible_group.offer_id IS NOT NULL AND offer_initiated_by = 'buyer' THEN order_gmv*0.01 END) AS eligible_buyer_offer_gmv"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date is not null AND eligible_group.offer_id IS NOT NULL AND offer_initiated_by = 'buyer' THEN eligible_group.order_id END) AS eligible_buyer_offer_orders"
    ,"COUNT(DISTINCT CASE WHEN eligible_group.book_date IS NOT NULL THEN eligible_group.user_id END)*1.00/COUNT(DISTINCT eligible_group.user_id) AS buyer_conversion_rate"
    ]
)

In [10]:
pre_all, post_all, _, _ = create_prepost(conn=my_conn,
                 control_segments=control_segments,
                 test_segments=test_segments,
                 pre_start_date = pre_start_date,
                 pre_end_date = pre_end_date,
                 post_start_date = post_start_date,
                 post_end_date = post_end_date,
                 segment_type='signup_segment',
                 standard_metrics=['BUYER_ORDERS','BUYER_OFFERS'],
                 adhoc_metrics=[eligible_group]
                                              )

2020-04-28 22:00:19,002| ERROR   | Password is required for key /Users/yingyingchu/.ssh/id_rsa
2020-04-28 22:01:33,442| ERROR   | Password is required for key /Users/yingyingchu/.ssh/id_rsa


Ratio Error: gmv per dau
Ratio Error: buyers per dau
Ratio Error: listers per dau
Ratio Error: listings per dau
Ratio Error: listings per listers
Ratio Error: gmv per dau
Ratio Error: buyers per dau
Ratio Error: listers per dau
Ratio Error: listings per dau
Ratio Error: listings per listers


In [11]:
ab_result_all=create_pretty_outputs(pre_all, 
                                    post_all, 
                                    max_pvalue=0.2,
                                    title='Abandon Cart Email Audience Expansion',
                                    lightspeed_only=True, 
                                    must_include_metrics=test_metrics)

,Metric,Pre Control Metric Mean,Pre Test Metric Mean,Pre Delta (%),Post Control Metric Mean,Post Test Metric Mean,Post Delta (%),PrePost Delta (%),PrePost Delta LCL (%),PrePost Delta UCL (%),p-value,Net Impact
0,buyers,"12,398.86","12,544.29",1.19%,"17,081.86","17,104.79",0.10%,-1.09%,-2.09%,-0.08%,0.0354,-185.705905
2,orders,"17,100.14","17,185.93",0.49%,"24,238.14","24,324.21",0.27%,-0.22%,-1.61%,1.17%,0.7471,-53.369835
7,gmv,"547,567.68","555,438.55",1.43%,"780,309.71","790,711.94",1.28%,-0.15%,-2.15%,1.85%,0.8795,-1161.717142
15,orders - buyer offer,"7,226.21","7,342.29",1.57%,"10,308.43","10,391.57",0.75%,-0.82%,-2.91%,1.28%,0.4294,-84.246467
16,gmv - buyer offer,"256,845.86","265,035.36",3.15%,"367,668.43","373,360.71",1.52%,-1.62%,-4.39%,1.15%,0.2381,-5960.413383
24,orders - buy now,"5,566.36","5,516.29",-0.92%,"7,883.79","7,842.93",-0.57%,0.35%,-1.61%,2.32%,0.7144,27.934822
30,gmv - buy now,"164,426.39","162,520.62",-0.81%,"237,326.00","239,765.87",1.11%,1.92%,-2.88%,6.73%,0.4173,4567.781051
48,eligible_users,"2,674.86","2,684.57",0.42%,"3,162.50","3,183.43",0.79%,0.37%,-1.88%,2.63%,0.7360,11.813845
49,eligible_buyers,168.93,161.14,-4.72%,223.50,223.50,0.78%,5.49%,-1.60%,12.59%,0.1224,12.274586
50,eligible_orders,229.50,214.79,-6.41%,309.86,312.57,1.71%,8.12%,0.72%,15.51%,0.0331,25.146007


In [12]:
ab_result_all_summary = create_pretty_outputs(pre_all, 
                        post_all, 
                        title='Abandon Cart Email Audience Expansion', 
                        lightspeed_only=True,
                        must_include_metrics =test_metrics,
                        diagnostic=False,
                        max_pvalue=0.2,
                        stats_list = ['Metric',
                                      #'Pre Control Metric Mean',
                                      #'Pre Test Metric Mean',
                                      #'Pre Delta (%)',
                                      #'Post Control Metric Mean',
                                      #'Post Test Metric Mean',
                                      #'Post Delta (%)',
                                      'PrePost Delta (%)',
                                      #'PrePost Delta LCL (%)',
                                      #'PrePost Delta UCL (%)',
                                      'p-value',
                                      #'Net Daily Impact',
                                     ])

,Metric,PrePost Delta (%),p-value
0,buyers,-1.09%,0.0354
2,orders,-0.22%,0.7471
7,gmv,-0.15%,0.8795
15,orders - buyer offer,-0.82%,0.4294
16,gmv - buyer offer,-1.62%,0.2381
24,orders - buy now,0.35%,0.7144
30,gmv - buy now,1.92%,0.4173
48,eligible_users,0.37%,0.7360
49,eligible_buyers,5.49%,0.1224
50,eligible_orders,8.12%,0.0331


In [13]:
from pmutils.knowledge import add_knowledge, NOTEBOOKNAME

add_knowledge( filename=NOTEBOOKNAME, # doesn't need to be changed
    authors = ['yingying'], # optional, defaults to Analytics
    extra_tags = ['abtest','comms'], # optional, defaults to None
    summary = 'Abandon Cart Email Audience Expansion', # optional, defaults to 'Click here for details',
    #image_assets=['']
)

Email not found in keyring. Please enter your poshmark email address: ········
Please confirm your poshmark email: ········


ValueError: Must be working inside analytics git repository